## Aggregating Data

Let us understand how to aggregate the data.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
val username = System.getProperty("user.name")

In [1]:
import org.apache.spark.sql.SparkSession

val username = System.getProperty("user.name")
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    config("spark.sql.warehouse.dir", s"/user/${username}/warehouse").
    enableHiveSupport.
    appName(s"${username} | Spark SQL - Basic Transformations").
    master("yarn").
    getOrCreate

username = itv002480
spark = org.apache.spark.sql.SparkSession@715c27b


org.apache.spark.sql.SparkSession@715c27b

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

* We can perform global aggregations as well as aggregations by key.
* Global Aggregations
  * Get total number of orders.
  * Get revenue for a given order id.
  * Get number of records with order_status either COMPLETED or CLOSED.
* Aggregations by key - using `GROUP BY`
  * Get number of orders by date or status.
  * Get revenue for each order_id.
  * Get daily product revenue (using order date and product id as keys).
* We can also use `HAVING` clause to apply filtering on top of aggregated data.
  * Get daily product revenue where revenue is greater than $500 (using order date and product id as keys).
* Rules while using `GROUP BY`.
  * We can have the columns which are specified as part of `GROUP BY` in `SELECT` clause.
  * On top of those, we can have derived columns using aggregate functions.
  * We cannot have any other columns that are not used as part of `GROUP BY` on derived column using non aggregate functions.
  * We will not be able to use aggregate functions or aliases used in the select clause as part of the where clause.
  * If we want to filter based on aggregated results, then we can leverage `HAVING` on top of `GROUP BY` (specifying `WHERE` is not an option)
* Typical query execution - FROM -> WHERE -> GROUP BY -> SELECT

In [2]:
%%sql
use itv002480_retail

Waiting for a Spark session to start...

++
||
++
++



In [3]:
%%sql

SELECT count(order_id) FROM orders

+---------------+
|count(order_id)|
+---------------+
|          68883|
+---------------+



In [4]:
%%sql

SELECT count(DISTINCT order_date) FROM orders

+--------------------------+
|count(DISTINCT order_date)|
+--------------------------+
|                       364|
+--------------------------+



In [5]:
%%sql

SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+



In [6]:
%%sql

SELECT count(1) 
FROM orders
WHERE order_status IN ('COMPLETE', 'CLOSED')

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [7]:
%%sql

SELECT order_date,
    count(1)
FROM orders
GROUP BY order_date

+--------------------+--------+
|          order_date|count(1)|
+--------------------+--------+
|2013-08-13 00:00:...|      73|
|2014-03-19 00:00:...|     130|
|2014-04-26 00:00:...|     251|
|2013-10-12 00:00:...|     162|
|2013-11-15 00:00:...|     135|
|2013-09-16 00:00:...|     121|
|2013-09-20 00:00:...|     139|
|2013-12-31 00:00:...|     266|
|2013-09-06 00:00:...|     276|
|2014-06-15 00:00:...|     128|
+--------------------+--------+
only showing top 10 rows



In [8]:
%%sql

SELECT order_status,
    count(1) AS status_count
FROM orders
GROUP BY order_status

+---------------+------------+
|   order_status|status_count|
+---------------+------------+
|PENDING_PAYMENT|       15030|
|       COMPLETE|       22899|
|        ON_HOLD|        3798|
| PAYMENT_REVIEW|         729|
|     PROCESSING|        8275|
|         CLOSED|        7556|
|SUSPECTED_FRAUD|        1558|
|        PENDING|        7610|
|       CANCELED|        1428|
+---------------+------------+



In [9]:
%%sql

SELECT order_item_order_id,
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY order_item_order_id LIMIT 10

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|              35450|       579.94|
|              35654|       449.96|
|              35800|       129.99|
|              35868|       529.93|
|              36106|       839.86|
|              36312|       479.94|
|              36655|       379.93|
|              36706|      1399.86|
|              36783|       824.95|
|              36982|       559.94|
+-------------------+-------------+



In [1]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

Waiting for a Spark session to start...

Magic sql failed to execute with error: 
Table or view not found: orders; line 1 pos 109

In [11]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >= 500
GROUP BY o.order_date,
    oi.order_item_product_id
LIMIT 10

            :  +- SubqueryAlias...


Magic sql failed to execute with error: 
cannot resolve '`revenue`' given input columns: [oi.order_item_product_price, oi.order_item_quantity, o.order_status, o.order_date, oi.order_item_subtotal, oi.order_item_product_id, o.order_customer_id, o.order_id, oi.order_item_id, oi.order_item_order_id]; line 1 pos 236;
'GlobalLimit 10
+- 'LocalLimit 10
   +- 'Aggregate ['o.order_date, 'oi.order_item_product_id], ['o.order_date, 'oi.order_item_product_id, 'round('sum('oi.order_item_subtotal), 2) AS revenue#137]
      +- 'Filter (order_status#141 IN (COMPLETE,CLOSED) && ('revenue >= 500))
         +- Join Inner, (order_id#138 = order_item_order_id#143)
            :- SubqueryAlias `o`
            :  +- SubqueryAlias `itv002480_retail`.`orders`
            :     +- HiveTableRelation `itv002480_retail`.`orders`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, [order_id#138, order_date#139, order_customer_id#140, order_status#141]
            +- SubqueryAlias `oi`
               +- SubqueryAl

In [12]:
%%sql

SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >= 500
LIMIT 10

+--------------------+------...


+--------------------+---------------------+-------+
|          order_date|order_item_product_id|revenue|
+--------------------+---------------------+-------+
|2014-04-09 00:00:...|                  191|6599.34|
|2014-05-29 00:00:...|                 1014|2698.92|
|2014-06-01 00:00:...|                 1073|3199.84|
|2014-06-30 00:00:...|                  502| 4000.0|
|2013-08-12 00:00:...|                  627| 3199.2|
|2013-09-04 00:00:...|                  957|3599.76|
|2013-10-19 00:00:...|                 1004|5599.72|
|2013-11-06 00:00:...|                  502| 3800.0|
|2014-02-13 00:00:...|                  403|4159.68|
|2014-03-06 00:00:...|                  191| 4999.5|
+--------------------+---------------------+-------+



* Using Spark SQL with Python or Scala

In [ ]:
spark.sql("SELECT count(order_id) FROM orders").show()

In [ ]:
spark.sql("SELECT count(DISTINCT order_date) FROM orders").show()

In [ ]:
spark.sql("""
SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2
""").show()

In [ ]:
spark.sql("""
SELECT count(1) 
FROM orders
WHERE order_status IN ('COMPLETE', 'CLOSED')
""").show()

In [ ]:
spark.sql("""
SELECT order_date,
    count(1)
FROM orders
GROUP BY order_date
""").show()

In [ ]:
spark.sql("""
SELECT order_status,
    count(1) AS status_count
FROM orders
GROUP BY order_status
""").show()

In [ ]:
spark.sql("""
SELECT order_item_order_id,
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY order_item_order_id
""").show()

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
""").show()

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND revenue >= 500
GROUP BY o.order_date,
    oi.order_item_product_id
""").show()

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
HAVING revenue >= 500
""").show()